In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X = pd.read_csv('train.csv', index_col='Id')
X_test_full = pd.read_csv('test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice              
X.drop(['SalePrice'], axis=1, inplace=True)


In [7]:
# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numeric columns
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

# One-hot encode the data (to shorten the code, we use pandas)
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

In [9]:
from xgboost import XGBRegressor

# Define the model
my_model_1 = XGBRegressor() # Your code here

# Fit the model
my_model_1.fit(X_train,y_train)


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

## Predictions
### Part B

In [11]:
from sklearn.metrics import mean_absolute_error


# Get predictions
predictions_1 = my_model_1.predict(X_valid)
predictions_1

array([180517.48 , 147601.61 , 110418.016, 215847.9  ,  91681.88 ,
       111870.805, 263483.72 , 148291.75 , 437694.56 , 161465.7  ,
       185297.62 , 136833.62 , 233315.   , 121832.3  , 117813.96 ,
       146678.56 , 224207.3  , 125697.555, 136710.3  , 192880.52 ,
       128928.86 , 143064.06 , 110618.305, 157551.58 , 188069.22 ,
       168270.64 , 180631.4  ,  72046.71 , 318782.62 , 101511.87 ,
       119958.695, 183278.23 , 140170.33 , 267998.06 , 350595.5  ,
       197427.33 , 303507.1  , 117701.734, 220137.88 , 316072.5  ,
       238313.42 , 125487.84 , 187199.5  , 300181.97 , 290329.   ,
       151480.44 , 123265.336, 131302.06 , 164329.92 ,  94760.18 ,
       424280.53 , 144024.98 , 172232.47 ,  77216.875, 219971.62 ,
       108312.39 , 144846.08 , 247667.4  , 141727.27 , 102231.555,
       128807.63 , 132450.42 , 148109.34 , 164187.42 , 201151.25 ,
       162853.72 , 131784.73 , 232565.34 , 120255.62 , 190808.64 ,
       183987.61 , 132344.03 ,  78886.01 , 248834.08 ,  95061.

In [13]:
# Calculate MAE
mae_1 = mean_absolute_error(predictions_1,y_valid) # Your code here

print("Mean Absolute Error:" , mae_1)


Mean Absolute Error: 17662.736729452055


## Step 2: Improve the model

In [14]:
my_model_2 = XGBRegressor(n_estimators=1000, learning_rate=0.05)

# Fit the model
my_model_2.fit(X_train, y_train)

# Get predictions
predictions_2 = my_model_2.predict(X_valid)

# Calculate MAE
mae_2 = mean_absolute_error(predictions_2, y_valid)
print("Mean Absolute Error:" , mae_2)

Mean Absolute Error: 16688.691513270547


## Step 3: Break the model
In this step, you will create a model that performs worse than the original model in Step 1. This will help you to develop your intuition for how to set parameters. You might even find that you accidentally get better performance, which is ultimately a nice problem to have and a valuable learning experience!

Begin by setting my_model_3 to an XGBoost model, using the XGBRegressor class. Use what you learned in the previous tutorial to figure out how to change the default parameters (like n_estimators and learning_rate) to design a model to get high MAE.
Then, fit the model to the training data in X_train and y_train.
Set predictions_3 to the model's predictions for the validation data. Recall that the validation features are stored in X_valid.
Finally, use the mean_absolute_error() function to calculate the mean absolute error (MAE) corresponding to the predictions on the validation set. Recall that the labels for the validation data are stored in y_valid.

In [15]:
# Define the model
my_model_3 = XGBRegressor(n_estimators=1)
# Fit the model
my_model_3.fit(X_train, y_train)

# Get predictions
predictions_3 = my_model_3.predict(X_valid)

# Calculate MAE
mae_3 = mean_absolute_error(predictions_3, y_valid)
print("Mean Absolute Error:" , mae_3)


Mean Absolute Error: 127895.0828807256
